# **1. Kết nối với google drive**

In [ ]:
# kết nối với gg drive
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NLP 4+1/BTL3- BERT and GPT/')
os.getcwd()

Mounted at /content/drive


'/content/drive/.shortcut-targets-by-id/1bs2wermM-w1hKEOcHyv-UmLHY5iag5cm/NLP 4+1/BTL3- BERT and GPT'

# **2. Install và import các thư viện cần thiết**

In [ ]:
!pip install keras_preprocessing --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 3.6 MB/s eta 0:00:00


In [ ]:
import re
import time
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None 

from keras_preprocessing.text import Tokenizer

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import pickle

# saving
def save_file(data, path):
    with open(path + '.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
def load_file(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

# **3. Set các biến cục bộ và load dữ liệu**

In [ ]:
MAXLEN_QUESTION = 32
MAXLEN_ANSWER = 64
EPOCHS = 200
BATCH_SIZE = 32
path_data = './dataset/'
path_pre_trained = './pre-trained/LSTM (200 epoch)/'

In [ ]:
df_train = pd.read_csv(path_data + 'train.csv')
df_train

,question,answer
0,what canbe the cause for chlamydia not going a...,i do not know how the chlamydia was treated wa...
1,can you overfeed an infant,i really do not have enough information to spe...
2,can you catch a cold through oral sex,hi i think your find its a hard thing to do un...
3,why do i get ring worm ever year ever year i r...,it is not a worm ringworm is just the lay term...
4,what ingredients should i look for in toothpaste,use fluoride toothpaste to help avoid cavities...
...,...,...
6260,is spinal meningitis contagious,yes some types of meningitis are contagious so...
6261,can exercise delay a period 20 days late past ...,hi women who exercise do tend to get skipped p...
6262,how do i care for my dental implant,dental implants require the same care as real ...
6263,can you get pregnant through underwear me and ...,no you have to have sex to get pregnant be car...


# **4. Xử lý dữ liệu**

## ***4.1 Thêm ký tự đầu câu và kết thúc câu cho đầu ra***

In [ ]:
df_train['answer'] = df_train['answer'].apply(lambda text: f'<s> {text} </s>')
df_train

,question,answer
0,what canbe the cause for chlamydia not going a...,<s> i do not know how the chlamydia was treate...
1,can you overfeed an infant,<s> i really do not have enough information to...
2,can you catch a cold through oral sex,<s> hi i think your find its a hard thing to d...
3,why do i get ring worm ever year ever year i r...,<s> it is not a worm ringworm is just the lay ...
4,what ingredients should i look for in toothpaste,<s> use fluoride toothpaste to help avoid cavi...
...,...,...
6260,is spinal meningitis contagious,<s> yes some types of meningitis are contagiou...
6261,can exercise delay a period 20 days late past ...,<s> hi women who exercise do tend to get skipp...
6262,how do i care for my dental implant,<s> dental implants require the same care as r...
6263,can you get pregnant through underwear me and ...,<s> no you have to have sex to get pregnant be...


# ***4.2 Khởi tạo bộ tokenizer***

In [ ]:
questions = list(df_train['question'].values)
answers = list(df_train['answer'].values)

In [ ]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(questions + answers)

# save
save_file(tokenizer, path_pre_trained + 'tokenizer') 

In [ ]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
print(f'Vocabulary size : {VOCAB_SIZE}')

Vocabulary size : 13231


# ***4.3 Tokenizer dữ liệu cho Encoder (questions)***

In [ ]:
# encoder
tokenized_questions = tokenizer.texts_to_sequences(questions)
encoder_inp = pad_sequences(tokenized_questions, maxlen=MAXLEN_QUESTION, padding='post')

print(encoder_inp.shape)
print(questions[0])
print(tokenized_questions[0])
print(encoder_inp[0])

(6265, 32)
what canbe the cause for chlamydia not going away after treatment and no sexual intercourse after being diagnosed
[25, 4871, 1, 44, 15, 514, 14, 331, 296, 68, 113, 8, 34, 384, 716, 68, 227, 282]
[  25 4871    1   44   15  514   14  331  296   68  113    8   34  384
  716   68  227  282    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


# 2.3 Tokenizer cho ..

In [ ]:
# decoder
tokenized_answers = tokenizer.texts_to_sequences(answers)
decoder_inp = pad_sequences(tokenized_answers, maxlen=MAXLEN_ANSWER, padding='post')

print(decoder_inp.shape)
print(answers[0])
print(tokenized_answers[0])
print(decoder_inp[0])

(6265, 64)
<s> i do not know how the chlamydia was treated was the dose appropriate and taken long enough were you compliant in taking it properly are you male or female do have more than one partner as much as i would like to address your question i really do not have enough information </s>
[6, 9, 26, 14, 86, 46, 1, 514, 85, 226, 85, 1, 406, 729, 8, 366, 125, 349, 410, 5, 5460, 18, 82, 13, 308, 17, 5, 449, 21, 388, 26, 19, 70, 129, 59, 621, 32, 204, 32, 9, 35, 67, 4, 945, 12, 229, 9, 134, 26, 14, 19, 349, 264, 7]
[   6    9   26   14   86   46    1  514   85  226   85    1  406  729
    8  366  125  349  410    5 5460   18   82   13  308   17    5  449
   21  388   26   19   70  129   59  621   32  204   32    9   35   67
    4  945   12  229    9  134   26   14   19  349  264    7    0    0
    0    0    0    0    0    0    0    0]


In [ ]:
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]
    
padded_answers = pad_sequences(tokenized_answers, maxlen=MAXLEN_ANSWER, padding='post')
decoder_final_output = to_categorical(padded_answers, VOCAB_SIZE)

print(decoder_final_output.shape)
print(tokenized_answers[0])
print(padded_answers[0])
print(decoder_final_output[0])

(6265, 64, 13231)
[9, 26, 14, 86, 46, 1, 514, 85, 226, 85, 1, 406, 729, 8, 366, 125, 349, 410, 5, 5460, 18, 82, 13, 308, 17, 5, 449, 21, 388, 26, 19, 70, 129, 59, 621, 32, 204, 32, 9, 35, 67, 4, 945, 12, 229, 9, 134, 26, 14, 19, 349, 264, 7]
[   9   26   14   86   46    1  514   85  226   85    1  406  729    8
  366  125  349  410    5 5460   18   82   13  308   17    5  449   21
  388   26   19   70  129   59  621   32  204   32    9   35   67    4
  945   12  229    9  134   26   14   19  349  264    7    0    0    0
    0    0    0    0    0    0    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
encoder_inp.shape, decoder_inp.shape, decoder_final_output.shape

((6265, 32), (6265, 64), (6265, 64, 13231))

# 3. Xây dựng và training model

In [ ]:
enc_inputs = Input(shape=(None,))
enc_embedding = Embedding(VOCAB_SIZE, 256, mask_zero=True)(enc_inputs)
_, state_h, state_c = LSTM(256, return_state=True)(enc_embedding)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_embedding = Embedding(VOCAB_SIZE, 256, mask_zero=True)(dec_inputs)
dec_lstm = LSTM(256, return_state=True, return_sequences=True)

dec_outputs, _, _ = dec_lstm(dec_embedding, initial_state=enc_states)
dec_dense = Dense(VOCAB_SIZE, activation='softmax')
output = dec_dense(dec_outputs)

In [ ]:
model = Model([enc_inputs, dec_inputs], output)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    3387136     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    3387136     ['input_2[0][0]']                
                                                                                              

In [ ]:
def generate_batch(encoder_inp_data, decoder_inp_data, padded_answer_data, batch_size = 32):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(encoder_inp_data), batch_size):
            encoder_input = encoder_inp_data[j:j+batch_size]
            decoder_input = decoder_inp_data[j:j+batch_size]
            decoder_answer = to_categorical(padded_answer_data[j:j+batch_size],VOCAB_SIZE)

            yield([encoder_input, decoder_input], decoder_answer)

In [ ]:
t0 = time.time()
history = model.fit(generate_batch(encoder_inp, decoder_inp, padded_answers, batch_size = BATCH_SIZE),
                    steps_per_epoch = len(encoder_inp)//BATCH_SIZE,
                    epochs = EPOCHS)
time_train = time.time() - t0

# save
model.save(path_pre_trained + 'model_LSTM.h5')

Epoch 1/200
195/195 [==============================] - 68s 247ms/step - loss: 6.9059
Epoch 2/200
195/195 [==============================] - 32s 162ms/step - loss: 6.5478
Epoch 3/200
195/195 [==============================] - 31s 160ms/step - loss: 6.5190
Epoch 4/200
195/195 [==============================] - 31s 158ms/step - loss: 6.4185
Epoch 5/200
195/195 [==============================] - 32s 163ms/step - loss: 6.3113
Epoch 6/200
195/195 [==============================] - 31s 159ms/step - loss: 6.1890
Epoch 7/200
195/195 [==============================] - 31s 158ms/step - loss: 6.0783
Epoch 8/200
195/195 [==============================] - 32s 164ms/step - loss: 5.9803
Epoch 9/200
195/195 [==============================] - 31s 159ms/step - loss: 5.8877
Epoch 10/200
195/195 [==============================] - 31s 159ms/step - loss: 5.7983
Epoch 11/200
195/195 [==============================] - 32s 163ms/step - loss: 5.7120
Epoch 12/200
195/195 [==============================] - 31s 159

In [ ]:
val_compare = {
    'loss_history': history.history['loss'],
    'time_train': time_train
}

# save
save_file(val_compare, path_pre_trained + 'val_compare')

In [ ]:
def make_inference_models():
    dec_state_input_h = Input(shape=(256,))
    dec_state_input_c = Input(shape=(256,))
    dec_states_inputs = [dec_state_input_h, dec_state_input_c]
    dec_outputs, state_h, state_c = dec_lstm(dec_embedding,
                                            initial_state=dec_states_inputs)
    dec_states = [state_h, state_c]
    dec_outputs = dec_dense(dec_outputs)
    dec_model = Model(
        inputs=[dec_inputs] + dec_states_inputs,
        outputs = [dec_outputs] + dec_states)
    print('Inference decoder:')
    dec_model.summary()
    print('Inference encoder:')
    enc_model = Model(inputs=enc_inputs, outputs=enc_states)
    enc_model.summary()
    return enc_model, dec_model

enc_model, dec_model = make_inference_models()

# save
enc_model.save(path_pre_trained + 'enc_model.h5')
dec_model.save(path_pre_trained + 'dec_model.h5')

Inference decoder:
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    3387136     ['input_2[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 256)]        0           []                               
                                                                         

In [ ]:
def str_to_tokens(sentence):
    words = sentence
    tokens_list = tokenizer.texts_to_sequences([words])
    return pad_sequences(tokens_list, maxlen=MAXLEN_QUESTION,padding='post')
    
def bot_give_answer(question):
    states_values = enc_model.predict(str_to_tokens(question), verbose=0)
    empty_target_seq = np.zeros((1,1))
    empty_target_seq[0,0] = tokenizer.word_index['<s>']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq]+states_values, verbose=0)
        sampled_word_index = np.argmax(dec_outputs[0,-1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                if word != '</s>':
                    decoded_translation += f'{word} '
                sampled_word = word

        if sampled_word == '</s>' or len(decoded_translation.split()) > MAXLEN_ANSWER:
            stop_condition = True
        empty_target_seq = np.zeros((1,1))
        empty_target_seq[0,0] = sampled_word_index
        states_values = [h,c]

    return decoded_translation

In [ ]:
bot_give_answer('does the caffeine in coffee cause facial wrinkles')

'there are no scientific studies to be concerned about the use of the over the counter antihistamines you should also use a prescription medication and prescription medication medication but it is not a common cold medicines used for allergies and are not recommended to take with food drug interactions between products containing allergies and other ingredients such as pneumonia drugs or spicy and also contain '